In [ ]:
import pandas as pd
from germansentiment import SentimentModel
import torch


In [ ]:
dataset = pd.read_csv("../data/processed/songs_complete_final.csv")


In [ ]:
model = SentimentModel()

In [ ]:
mps_device = torch.device("mps")
model.model.to(mps_device)
model.device = "mps"

In [ ]:
classes, probabilities = model.predict_sentiment(dataset["lyrics"][:10], output_probabilities = True)


In [ ]:
from pprint import pprint
pprint(list(zip(classes, probabilities, dataset["lyrics"][:10])))

### Fine Tune Classifier

In [ ]:
import pandas as pd
ds = pd.read_csv("../data/processed/songs_complete_final.csv")
ds.dropna(subset=["genre_cat"], inplace=True)
ds["genre_cat"].unique()
factorized = pd.factorize(ds["genre_cat"])
print(factorized)
ds["labels"] = factorized[0]
ds.to_csv("../data/processed/songs_complete_final_no_nan.csv", index=False)

In [ ]:
from datasets import load_dataset
hf_dataset = load_dataset("csv", data_files="../data/processed/songs_complete_final_no_nan.csv")
# hf_dataset=hf_dataset.rename_columns({"genre_cat": "labels"})
print(hf_dataset)

In [ ]:
split = hf_dataset["train"].train_test_split(test_size=0.2)
train_ds = split["train"]
test_ds = split["test"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["lyrics"], padding="max_length", truncation=True)

In [ ]:
train_tokenized = train_ds.map(tokenize_function, batched=True)
test_tokenized = test_ds.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = train_tokenized.shuffle(seed=42).select(range(1000))
small_test_dataset = test_tokenized.shuffle(seed=42).select(range(100))

In [ ]:
id2label = {idx: val for idx, val in enumerate(factorized[1])}
print(id2label)
label2id = {val: key for (key, val) in id2label.items()}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=4, id2label=id2label, label2id=label2id)
# model.to("mps")

In [ ]:
training_args = TrainingArguments(
    output_dir="bert_genre",
    use_mps_device=True,
    num_train_epochs=2,
    # learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    # weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",)

In [ ]:
import numpy as np
import evaluate

accuraracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    results = {}
    results.update(accuraracy.compute(predictions=predictions, references=labels))
    results.update(recall.compute(predictions=predictions, references=labels, average="micro"))
    results.update(precision.compute(predictions=predictions, references=labels, average="micro"))
    results.update(f1.compute(predictions=predictions, references=labels, average="micro"))
    return results

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, return_all_scores=True, device="mps")



In [ ]:
toc1 = """
ch denke an das, was Du empfiehlst:
"Talent borrows, genius steals"
An das was Du mir anvertraust:
"Wähle dir deine fallen aus"
Ich glaub ich kanns erst jetzt verstehen
Wir müssen durch den Spiegel gehen
Danke auch für den Versuch
Das hier ist kein Wörterbuch
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
Völker! Auf zum Gefecht!
Die Illusion wird Menschenrecht
Ich bin nicht allein in meiner Sucht
Vor den Spießern auf der Flucht
Du denkst an mich
Ich denk' an Dich
Gegen die Welt
Gegen den Strich
Unser Denken
Verbündet sich
Gegen die Welt
Gegen den Strich
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Jeden Morgen, jede Nacht
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
Du streichst mir über
Mein Gesicht
Gegen die Welt
Gegen den Strich
Meine Liebe
Dein Verzicht
Gegen die Welt
Gegen den Strich
"""

In [ ]:
kool1 = """Ich komm auf die Bühne leg Hand ans Mic
Fang langsam an und demolier dich am Ende wie Van Damme
Geh hol deine Gun dann, wärs fair
Doch bis dahin ist das einzge was du gewinnst nur Land man
Guck euer Gerede macht Müde wie der Sandmann
Ob Türke, Russe oder der Rest von Grand Cannes
Bei jedem finden diese Tracks Anklang, aber was ist das
Deine Bitch hat ein Loch wie'n Schuss von ner Pumpgun
Na dann viel Fun man, ich an deiner Stelle würd
Sie am randstehen lassen wie'n Wandschrank
Egal kein tam-tam, wahrt den Anstand meine
Texte ficken alles wie Matrosen auf Landgang
Für mich ist rappen wie wandern, s läuft
Und ich bediene gerne euer Team alleine die Bank lang
Danke an alle die mein Talent verkannt ham
Heute seht ihr wohl alles anders wie in nem Handstand, oder nicht?
Nie, niemals kriegt ihr mich weg, nie wi-ieder geh' ich
Keiner von euch kann's, wie di-ieser Typ ist da Vinci
Und das hier ist meine Mona-Lisa
Des Original, die Eins, vielmals von ihm kopiert
Aber niemals erreicht, ihr meint ihr seids
Doch kommt nicht ran an Mona-Lisa
Ich renn' durch Beats wie'n Athlet
Ich nehm' Wörter und Silben und kann sie im Takt dreh'n
Des ist'n Fetisch, ähnlich wie bei Leuten, die auf Leder und Lack stehen
Ich kann mich an meinen Texten nicht satt sehen
Ich hab' den übelsten dich-glatt-bügelnden Flow
Alles, was ich rappe, muss "Zack Zack Zack" gehen
Sie wollen mein Album mehr, als Miss Milian nackt sehen
Feature mit dir, ich muss ablehnen
Was geht, alle nenn'n dich Kackface
Als würde dein Kopf unmittelbar hinter dem Sack kleben
Mich will jeder wie 'ne Wiese plattmähen
Mich battlen lass', wie bei 'nem Liebesakt blähen
"""

In [ ]:
pipe(kool1)